In [ ]:
import logging
from dateutil import parser
from datetime import datetime, timedelta, timezone


from rich import print
import pandas as pd
from pandas import Series
from pandas import DataFrame as DF
import plotly.express as px

from core.s3_utils import S3_Bucket
from core.constants import *


In [ ]:
#based on the whole list of vin we should activate we want to see which one or activated, deactivated, never activated 
fleet_info = pd.read_csv("../ayvens/fleet_info.csv", usecols=["VIN"], dtype={"Make":"string"})

activation_df = (
    fleet_info
    .rename(columns={"VIN": "vin"})
    .set_index("vin", drop=False)
)
# Add a new column 'activation' with value 'deactivated' for all rows
activation_df['activation'] = 'never activated'


In [ ]:
bucket = S3_Bucket()
keys = {}
all_vins = {}
# List all brand folders in the processed_ts directory
all_brands = bucket.list_subfolders(f"response/")
# all_brands = [brand.strip('/').split('/')[-1] for brand in all_brands if brand.endswith('/')]
print(all_brands)


In [ ]:
#We want to evaluate the activation status of the vehicle: activated or deactivated

bucket = S3_Bucket()
keys = {}
all_vins = {}

yesterday = datetime.now(timezone.utc) - timedelta(days=1)

for brand in all_brands:
    vins = bucket.list_subfolders(f"response/{brand}")
    for vin in vins:
        folders = bucket.list_subfolders(f"response/{brand}/{vin}/")
        path = f"response/{brand}/{vin}/"
        if 'temp' in folders:
            activation_df.loc[vin, "activation"] = "activated"
        else:
            activation_df.loc[vin, "activation"] = "deactivated"

activation_df.tail(10)

# # Display the results
# print("\nSample of VINs and their activation status:")
# print(activation_df.head(10))

# print("\nActivation status summary:")
# print(activation_df['Activation'].value_counts())

# Optional: Save the DataFrame to a CSV file
activation_df.to_csv("activation_status.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
max_odo_dict = {}
for brand, brand_keys in keys.items(): 
    brand_max_odos:Series = brand_keys["key"].apply(lambda key: bucket.read_parquet_df(key)["odometer"].max())
    max_odo_dict[brand] = brand_max_odos

In [ ]:
max_odos = pd.concat(max_odo_dict, keys=max_odo_dict.keys(), names=["brand"]).reset_index(0, drop=False).rename(columns={"key": "odometer"})
max_odos

In [ ]:
px.histogram(max_odos, color="brand", opacity=0.6)